<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/Multi_DOC_RAG_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install langchain-community==0.2.15
!pip install langchain-chroma==0.1.3
!pip install langchain-huggingface==0.0.3
!pip install langchain-groq==0.1.9
!pip install unstructured==0.15.0
!pip install unstructured[pdf]==0.15.0

In [7]:
!pip install langchain-text-splitters==0.2.2
!pip install nltk==3.8.1

In [8]:
!apt-get install popper-utils
!apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package popper-utils
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [9]:
import os

from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [26]:
import json
config_data = json.load(open("/content/config.json"))

GROQ_API_KEY = config_data["GROQ_API_KEY"]

#save the api key to environment variable
os.environ["GROQ_API_KEY"] = GROQ_API_KEY


In [11]:
loader=DirectoryLoader('data/',glob='*.pdf',loader_cls=UnstructuredPDFLoader)
documents=loader.load()

In [16]:
print((documents[2]))

page_content='The Wonders of Quantum Computing

Quantum computing represents a paradigm shift in the world of computation, promising unprecedented computational power and capabilities. At its core lies the principles of quantum mechanics, where quantum bits or qubits replace classical bits, allowing for the exploitation of quantum phenomena such as superposition and entanglement.

Superposition enables qubits to exist in multiple states simultaneously, exponentially increasing the computational possibilities. This property allows quantum computers to explore numerous potential solutions to a problem simultaneously, vastly outperforming classical computers for certain tasks. Furthermore, entanglement enables qubits to be correlated in such a way that the state of one qubit instantaneously affects the state of another, regardless of the distance between them.

One of the most promising applications of quantum computing is in the field of cryptography. Quantum computers have the potential

In [17]:
text_splitter=CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=500
    )
text_chunks=text_splitter.split_documents(documents)

In [18]:
persist_directory="doc_chroma_db"

In [19]:
embedding=HuggingFaceEmbeddings()

<ipython-input-19-8b002df105c1>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding=HuggingFaceEmbeddings()
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
vectors_store=Chroma.from_documents(
    documents=text_chunks,
    embedding=embedding,
    persist_directory=persist_directory
)

In [21]:
retriever=vectors_store.as_retriever()

In [27]:
llm=ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)

In [28]:
qa_chain=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [29]:
query="What does the document say about DELINEATING THE CLINICAL SYNDROME"
response=qa_chain.invoke({"query":query})

In [30]:
print(response)

{'query': 'What does the document say about DELINEATING THE CLINICAL SYNDROME', 'result': 'The document states that it is useful to divide Respiratory Tract Infections (RTIs) into those involving the upper and the lower tracts. \n\nThe clinical syndromes involving the upper tract include:\n\n1. Otitis media\n2. Mastoiditis\n3. Sinusitis\n4. Pharyngitis\n\nThe clinical syndromes involving the lower tract can be divided into:\n\n1. Tracheobronchitis\n2. Bronchiolitis\n3. Pneumonia\n\nMost of these conditions can exist in acute and chronic forms.', 'source_documents': [Document(metadata={'source': 'data/paper1.pdf'}, page_content='DELINEATING THE CLINICAL SYNDROME\n\nIt is useful to divide RTIs into those involving the upper and the lower tracts. The clinical syndromes involving the upper tract include otitis media, mastoiditis, sinusitis and pharyngitis, while infections of the lower tract can be divided into tracheobronchitis, bronchiolitis and pneumo- nia. Most of these conditions can 

In [32]:
print(response['result'])

The document states that it is useful to divide Respiratory Tract Infections (RTIs) into those involving the upper and the lower tracts. 

The clinical syndromes involving the upper tract include:

1. Otitis media
2. Mastoiditis
3. Sinusitis
4. Pharyngitis

The clinical syndromes involving the lower tract can be divided into:

1. Tracheobronchitis
2. Bronchiolitis
3. Pneumonia

Most of these conditions can exist in acute and chronic forms.


In [34]:
print("source is:",response['source_documents'][0].metadata["source"])

source is: data/paper1.pdf


In [35]:
query="What does the document say about Receptor Binding domain?"
response=qa_chain.invoke({"query":query})
print(response['result'])
print("source is:",response['source_documents'][0].metadata["source"])

The document discusses the Receptor Binding Domain (RBD) of the SARS-CoV-2 spike protein in several sections. Here are the key points:

1. **Location and importance**: The RBD is located on the surface of the spike protein and is crucial for the virus to bind to the host cell receptor, ACE2. Mabs that target the RBD can block ACE2 binding and neutralize the virus.
2. **Binding sites**: The RBD has several binding sites for antibodies, including the "neck and shoulders" region, which is the most potent area for neutralization. Mabs that bind to this region can block ACE2 binding and neutralize the virus.
3. **Conserved regions**: The RBD has two large continuous conserved regions, one starting from the front of the left shoulder and running down to the left flank, and another at the bottom part of the back of the RBD. Mabs that bind to these regions can neutralize the virus, but may not block ACE2 binding.
4. **Mutations and escape**: The RBD is subject to mutations, which can lead to a